In [3]:
import ee
ee.Authenticate()

Enter verification code:  4/1Adeu5BVpGK-3wy_Olt83dvK1DhGLmBzayjWpbbvxyyovBj2TkzsUbfCKHZk



Successfully saved authorization token.


In [4]:
import ee
import geemap
# metodo de clasificacion
import jenkspy

import geopandas as gpd
from geopandas.tools import overlay
from shapely.geometry import Polygon

import pandas as pd
import numpy as np

import openpyxl

In [5]:
ee.Initialize()

In [6]:
ruta_catastro='projects/ee-bismarksr17/assets/LOTES_CANHA_ENE'
ruta_ndvi = 'projects/ee-bismarksr17/assets/NDVI_01_2023'

CODIGO_PROPIEDAD='unidad_01'
NOM_PROPIEDAD='unidad_02'
CODIGO_CANHERO='unidad_03'
NOM_CANHERO='unidad_04'

VARIEDAD='variedad'
ESTADO='RENOVACION'
SUPERFICIE='area'

In [7]:
def asig_cat(label):
    if label == 1:
        return 0
    else:
        return label * 10

vis_params_caña = {
    'color': 'red', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

# parametro de visualizacion LOTES RENOVACION
vis_params_renov = {
    'color': 'blue', 
    'width': 2,
    'lineType': 'solid',
    'fillColor': '00000000',
}

In [8]:
ruta_fases = 'codigos_feb.xlsx'

In [9]:
df = pd.read_excel(ruta_fases)

FileNotFoundError: [Errno 2] No such file or directory: 'codigos_feb.xlsx'

In [8]:
df.head()

,unidad_01
0,1
1,2
2,3
3,4
4,5


In [44]:
lista_props = list(df['unidad_01'])

In [45]:
len(lista_props)

1416

In [46]:
print(lista_props)

[1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 36, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 50, 54, 55, 57, 61, 62, 63, 66, 67, 68, 71, 72, 74, 76, 78, 79, 83, 84, 85, 89, 92, 93, 94, 98, 100, 103, 104, 105, 106, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 122, 123, 125, 126, 127, 128, 129, 132, 133, 135, 137, 139, 140, 141, 142, 145, 146, 147, 149, 150, 155, 156, 159, 161, 162, 164, 168, 171, 173, 174, 176, 178, 179, 182, 183, 184, 185, 186, 187, 188, 189, 195, 197, 198, 201, 205, 206, 209, 211, 212, 213, 216, 217, 218, 219, 221, 222, 225, 226, 227, 228, 229, 230, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 246, 247, 249, 250, 251, 253, 256, 257, 258, 259, 260, 261, 263, 264, 265, 267, 270, 271, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 306, 307, 309, 310, 311, 313, 314, 318, 320, 324, 325, 326, 327

In [53]:
#lista_props = [682, 706, 707, 729, 708, 709, 710, 693, 692, 679, 678, 704, 702, 703, 127, 683, 694, 698, 696, 677, 697, 695, 699, 449, 700, 687, 725, 726, 891, 834, 833, 771, 839, 838, 802, 863, 953, 925, 690, 685, 681, 5, 916, 889, 890, 892, 753, 724, 723, 1005, 1006, 747, 746, 745, 1016, 728, 887, 764, 765, 763, 762, 1049, 715, 714, 772, 1050, 1051, 1052, 1053, 744, 936, 835, 855, 228, 886, 742, 1054, 734, 755, 754, 776, 46, 757, 775, 774, 720, 722, 721, 719, 718, 717, 139, 147, 733, 732, 1819, 750, 877, 617, 741, 1482, 1485, 1481, 1480, 1483, 1486, 1484, 591, 216, 55, 217, 256, 219, 218, 205, 752, 187, 26, 910, 909, 1089, 830, 779, 780, 350, 766, 859, 894, 852, 1097, 848, 849, 971, 853, 840, 843, 842, 846, 845, 844, 782, 904, 275, 854, 630, 631, 778, 613, 141, 142, 767, 1099, 773, 1461, 1551, 803, 954, 386, 384, 861, 932, 952, 951, 950, 770, 769, 768, 864, 930, 862, 810, 811, 445, 812, 804, 805, 807, 806, 814, 827, 860, 816, 815, 813, 809, 817, 866, 11, 263, 865, 348, 898, 896, 969, 913, 132, 1139, 716, 897, 900, 901, 899, 478, 414, 868, 871, 869, 878, 873, 874, 799, 800, 273, 758, 759, 289, 133, 783, 784, 365, 785, 629, 201, 777, 258, 522, 902, 888, 893, 924, 923, 912, 788, 796, 789, 790, 794, 792, 793, 879, 787, 832, 921, 989, 920, 988, 915, 883, 994, 882, 880, 944, 1022, 1094, 93, 135, 355, 1220, 1252, 928, 922, 24, 935, 927, 918, 473, 472, 942, 937, 938, 939, 9, 947, 1411, 970, 1242, 400, 22, 419, 415, 964, 963, 966, 956, 957, 958, 968, 399, 962, 1015, 985, 986, 984, 980, 973, 176, 976, 975, 974, 979, 982, 981, 993, 991, 416, 418, 417, 961, 1033, 1024, 1025, 1034, 1036, 1035, 1028, 1026, 1009, 1013, 1086, 1085, 1007, 1011, 1008, 1010, 233, 1018, 1020, 1023, 1095, 1057, 1047, 1046, 1066, 168, 1072, 1073, 1074, 1075, 377, 378, 1076, 1077, 1079, 1078, 1080, 1083, 1084, 1087, 1042, 1045, 1092, 1039, 1093, 1041, 1043, 1040, 264, 238, 705, 1122, 1123, 1124, 1126, 1121, 1118, 1125, 1120, 1116, 1117, 1115, 1113, 1114, 1119, 1146, 1130, 1132, 1128, 1142, 1145, 1140, 1149, 1151, 1156, 1157, 1154, 1158, 43, 1160, 1163, 1161, 1162, 541, 396, 395, 1173, 1182, 1180, 1183, 1184, 1177, 1181, 1178, 1176, 1179, 1185, 1171, 1165, 1168, 1186, 990, 1216, 1193, 1197, 1208, 1207, 1204, 1206, 1200, 1203, 1205, 1195, 1211, 1199, 1194, 1212, 1201, 156, 36, 1214, 1198, 1227, 1234, 1226, 1247, 1249, 1248, 1233, 1241, 1240, 1228, 1229, 1231, 1232, 1237, 1243, 1244, 1219, 1221, 1222, 903, 1292, 1294, 1274, 1291, 1287, 1290, 1283, 1286, 1285, 1264, 1267, 1266, 1265, 1268, 1282, 1262, 1254, 1258, 1263, 1259, 1257, 1256, 943, 54, 286, 287, 198, 279, 1313, 1314, 1306, 1307, 1308, 1311, 1305, 1302, 1309, 1303, 1304, 450, 444, 1299, 1301, 63, 1350, 1349, 62, 556, 1335, 8, 340, 1688, 250, 1170, 189, 466, 568, 567, 1321, 23, 47, 50, 324, 376, 559, 558, 510, 278, 1210, 1374, 1373, 1372, 1380, 1384, 1376, 1377, 1378, 1381, 1368, 1379, 1385, 1371, 1370, 1386, 1375, 1382, 1389, 1394, 1352, 1356, 1351, 1346, 1344, 1347, 1342, 1354, 1364, 1365, 1366, 1362, 1343, 361, 1359, 1358, 1324, 1326, 1327, 1330, 1333, 1338, 1339, 1340, 1341, 1332, 1322, 1337, 1336, 1331, 1100, 1397, 442, 1408, 435, 434, 1404, 1399, 1409, 1406, 1403, 1407, 1405, 1402, 1401, 1413, 1412, 1420, 1419, 1418, 1416, 1417, 616, 539, 1238, 781, 1422, 575, 1425, 1421, 674, 1430, 1433, 1438, 328, 1451, 1426, 1424, 668, 360, 1452, 249, 547, 669, 1443, 1450, 1459, 185, 1456, 1473, 1472, 1475, 1474, 1489, 1476, 326, 1478, 1494, 1495, 424, 1498, 1499, 246, 1492, 1491, 405, 596, 597, 598, 1021, 1111, 1502, 1511, 115, 197, 1269, 1367, 1388, 1393, 1431, 1432, 1440, 1446, 1457, 1464, 1465, 1469, 1470, 1497, 1505, 30, 1513, 1518, 1520, 211, 1392, 1537, 1536, 1526, 1573, 1545, 155, 1579, 76, 244, 1516, 1544, 1546, 1530, 945, 1548, 1562, 1567, 1566, 1512, 1568, 1554, 1447, 1556, 1557, 1558, 1561, 1550, 1539, 1563, 1564, 570, 1088, 1569, 633, 1190, 1570, 1538, 908, 1522, 1523, 691, 1559, 1524, 1549, 1525, 1540, 1541, 1553, 1578, 1547, 1552, 492, 1487, 1414, 525, 1225, 1815, 620, 1542, 1323, 1467, 1818, 749, 1820, 136, 934, 411, 1585, 42, 476, 1587, 1275, 1588, 230, 39, 72, 519, 1235, 867, 1598, 1597, 1596, 1600, 1601, 1593, 1602, 801, 1580, 1609, 1610, 184, 1490, 150, 1614, 1611, 1613, 140, 1353, 343, 836, 1617, 1616, 1620, 1621, 1622, 1623, 495, 391, 1627, 1626, 1625, 1624, 1630, 1629, 1628, 536, 1619, 406, 1633, 1639, 534, 1640, 1644, 1643, 15, 1651, 1652, 1656, 1658, 1655, 179, 1660, 1661, 1663, 730, 1671, 1673, 1677, 1676, 1675, 1674, 375, 1682, 1683, 229, 1684, 260, 1685, 105, 128, 188, 103, 261, 870, 1460, 1091, 1325, 1607, 1355, 1672, 1535, 1649, 1345, 1638, 1605, 3, 1668, 756, 1635, 1648, 603, 621, 1645, 10, 1631, 57, 1665, 1250, 1659, 1662, 1606, 1669, 83, 1667, 1014, 1213, 1670, 1646, 1642, 159, 1687, 1686, 1641, 1612, 825, 1690, 331, 1153, 49, 1608, 1510, 206, 234, 186, 66, 38, 2, 1694, 113, 369, 941, 791, 31, 67, 68, 1509, 146, 109, 162, 164, 926, 111, 61, 1710, 1697, 114, 1699, 21, 1817, 1816, 265, 174, 222, 949, 624, 1708, 1709, 1716, 1215, 332, 1574, 1315, 295, 1736, 1738, 1737, 851, 907, 1739, 737, 1741, 1740, 1742, 1743, 1166, 1458, 149, 1745, 1746, 1750, 1749, 1751, 1752, 931, 1357, 1757, 345, 1453, 1759, 1760, 1761, 408, 1586, 1764, 1766, 1768, 1189, 1771, 1781, 1780, 1131, 307, 872, 1793, 1755, 1528, 1693, 1790, 1765, 1762, 1726, 1719, 875, 1721, 1696, 1744, 1735, 84, 1767, 1260, 1732, 1653, 1695, 1730, 1733, 1788, 1775, 1714, 1778, 1720, 1782, 1724, 1701, 1783, 1791, 235, 1712, 527, 1731, 1722, 1702, 1703, 1772, 1786, 12, 1594, 992, 1792, 1774, 1773, 1776, 1794, 831, 1038, 1277, 1279, 1280, 1281, 1278, 318, 1784, 33, 1789, 1797, 1159, 1592, 1718, 1711, 1770, 1796, 1799, 1795, 929, 1706, 1805, 1806, 1800, 1802, 44, 1798, 1255, 1251, 1801, 1748, 1637, 500, 490, 178, 1284, 1503, 623, 299, 1808, 1807, 1441, 1812, 1310, 761, 1436, 227, 1758, 1824, 1850, 1846, 1827, 1777, 1828, 1829, 1832, 1844, 1845, 1847, 1848, 1849, 1851, 1853, 578, 1833, 32, 1854, 1826, 247, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1866, 1865, 592, 1867, 1868, 426, 1488, 1763, 739, 1869, 1870, 601, 1823, 1700, 1872, 1874, 1873, 1876, 1875, 1877]
lista_props = [1040]

In [54]:
len(lista_props)

1

In [36]:
df_intersects = gpd.read_file('INTERSECT.shp')

In [55]:
contador = 0
for cod_prop in lista_props:
    print('inicio:', cod_prop)
    propiedad = ee.FeatureCollection(ruta_catastro)\
                .filter(ee.Filter.eq(CODIGO_PROPIEDAD, cod_prop))
    lotes_canha = propiedad.filter(ee.Filter.neq(VARIEDAD, ESTADO))
    lotes_renov = propiedad.filter(ee.Filter.eq(VARIEDAD, ESTADO))
    NDVI = ee.Image(ruta_ndvi)
    NDVI_clip = NDVI.clip(lotes_canha.geometry())
    # crea un sample de los valore de pixel del NDVI
    NDVI_values = NDVI_clip.sampleRegions(lotes_canha.geometry())
    # reduce el resultado a valores de NDVI
    pixel_values = NDVI_values.reduceColumns(ee.Reducer.toList(),['b1']).get('list').getInfo()
    # aplica metodo Jenks
    breaks = jenkspy.jenks_breaks(pixel_values, n_classes=8)
    NDVI_class = ee.Image(-1).where(NDVI.lt(breaks[1]),1)\
                        .where(NDVI.gte(breaks[1]),2)\
                        .where(NDVI.gte(breaks[2]),3)\
                        .where(NDVI.gte(breaks[3]),4)\
                        .where(NDVI.gte(breaks[4]),5)\
                        .where(NDVI.gte(breaks[5]),6)\
                        .where(NDVI.gte(breaks[6]),7)\
                        .where(NDVI.gte(breaks[7]),8)
    NDVI_class_clip = NDVI_class.clip(lotes_canha.geometry())
    clasify = NDVI_class_clip.reproject(crs="EPSG:32720", scale=10)
    vector = clasify.reduceToVectors(**{
        'geometry': lotes_canha.geometry(),
        'crs': clasify.projection(),
        'scale': 10,
        'geometryType': 'polygon',
        'eightConnected': False
    })
    
    DF_VECTOR = vector
    
    lotes_local = geemap.ee_to_geopandas(lotes_canha)
    
    lista = vector.toList(vector.size()).getInfo()
    lista_vector=[]
    for item in lista:
        dic = {'geometry':Polygon(item['geometry']['coordinates'][0]), 'count':item['properties']['count'], 'label':item['properties']['label']}
        lista_vector.append(dic)
    
    vector_local = gpd.GeoDataFrame(lista_vector)
    
    intersect = overlay(lotes_local, vector_local, how="intersection")
    intersect.crs = "EPSG:4326"
    intersect = intersect.to_crs(epsg=32720)
    intersect['area_2'] = intersect['geometry'].area/10000
    
    area_01 = intersect['area_2'].sum()
    area_02 = lotes_local[SUPERFICIE].sum()
    area_diff = (area_02 - area_01)/len(intersect)
    intersect['area_2'] = intersect['area_2'] + area_diff
    
    DF = intersect.copy()
    dina = pd.pivot_table(DF, values='area_2', index=[CODIGO_PROPIEDAD, NOM_PROPIEDAD, CODIGO_CANHERO, NOM_CANHERO, 'label'], aggfunc=np.sum)
    dina2 = dina.reset_index()
    
    dina2['tch'] = dina2['label'].apply(asig_cat)
    
    # Cargar el archivo de Excel existente
    wb = openpyxl.load_workbook('DATA_EST.xlsx')
    # Seleccionar la hoja de trabajo
    ws = wb['data']
    
    for i in range(0, len(dina2)):
        new_row = [dina2.iloc[i,0], dina2.iloc[i,1], dina2.iloc[i,2], dina2.iloc[i,3], dina2.iloc[i,4], dina2.iloc[i,5], dina2.iloc[i,6]]
        ws.append(new_row)
    wb.save('DATA_EST.xlsx')
    
    df_intersects = pd.concat([df_intersects, DF], ignore_index=True)
    
    contador = contador + 1 
    print('fin: ', cod_prop)
    print('___________________CONTADOR: ', contador)
    
print('FIN......!')

inicio: 1040
fin:  1040
___________________CONTADOR:  1
FIN......!


In [56]:
len(df_intersects)

974024

In [58]:
df_intersects.to_file("INTERSECT_2.shp", driver="ESRI Shapefile")

In [57]:
df_intersects['area_2'].sum()

52725.40608883015

In [32]:
Map = geemap.Map(basemap="SATELLITE")

In [92]:
Map.centerObject(propiedad.geometry(), 14)
Map.addLayer(lotes_canha.style(**vis_params_caña), {}, 'Lotes Caña')
Map.addLayer(lotes_renov.style(**vis_params_renov), {}, 'Lotes Renovación')

In [33]:
Map.centerObject(propiedad.geometry(), 14)
Map.addLayer(lotes_canha.style(**vis_params_caña), {}, 'Lotes Caña')
Map.addLayer(lotes_renov.style(**vis_params_renov), {}, 'Lotes Renovación')

In [34]:
Map

Map(center=[-17.240370201942387, -63.1623575923626], controls=(WidgetControl(options=['position', 'transparent…

In [124]:
categoria = ['Nulo','Muy Bajo','Bajo Medio','Bajo','Medio','Medio Alto','Alto','Muy Alto']
colores = ['','','','','','','','']